In [2]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, regexp_extract, coalesce

In [3]:
spark = SparkSession.builder \
    .appName("Notebook Glue Job Simulation") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/01/29 19:26:17 WARN Utils: Your hostname, 01120230807W11I resolves to a loopback address: 127.0.1.1; using 172.24.33.137 instead (on interface eth0)
25/01/29 19:26:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/29 19:26:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
json_path = "../data/skyscanner_info_example.json"
df = spark.read.option("multiline", "true").json(json_path)


In [18]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- context: struct (nullable = true)
 |    |    |-- filterTotalResults: long (nullable = true)
 |    |    |-- sessionId: string (nullable = true)
 |    |    |-- status: string (nullable = true)
 |    |    |-- totalResults: long (nullable = true)
 |    |-- destinationImageUrl: string (nullable = true)
 |    |-- filterStats: struct (nullable = true)
 |    |    |-- airports: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- airports: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- entityId: string (nullable = true)
 |    |    |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- city: string (nullable = true)
 |    |    |-- carriers: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |   

In [19]:
itineraries_df = df.selectExpr(
    "ori",
    "dest",
    "dest",
    "year",
    "month",
    "week",
    "day",
    "data.itineraries"
    )

# itineraries_df = df.select(
#     "data.context.sessionId", 
#     "data.context.status", 
#     "data.itineraries"
#     )
itineraries_df = itineraries_df.withColumn("itinerary", explode(col("itineraries")))

In [23]:
from pyspark.sql.functions import col, explode

# Seleccionar columnas relevantes del itinerario
details_df = itineraries_df.select(
    "ori", "dest", "year", "month", "week", "day",
    col("itinerary.price.raw").alias("price_raw"),
    col("itinerary.legs").alias("legs"),
    col("itinerary.farePolicy").alias("farePolicy"),
    col("itinerary.isSelfTransfer").alias("isSelfTransfer"),
    col("itinerary.isProtectedSelfTransfer").alias("isProtectedSelfTransfer")
)

# Explotar legs para dividir los vuelos en filas individuales
legs_df = details_df.withColumn("leg", explode("legs")) \
    .withColumn("marketing", explode("leg.carriers.marketing")) \
    .withColumn("segments", explode("leg.segments"))

# Definir columnas finales de salida
selected_columns = [
    "ori", "dest", "month", "week", "day", "price_raw",
    col("leg.id").alias("leg_id"),
    col("leg.origin.name").alias("origin_name"),
    col("leg.origin.displayCode").alias("origin_airport"),
    col("leg.origin.city").alias("origin_city"),
    col("leg.origin.country").alias("origin_country"),
    col("leg.destination.name").alias("destination_name"),
    col("leg.destination.displayCode").alias("destination_airport"),
    col("leg.destination.city").alias("destination_city"),
    col("leg.destination.country").alias("destination_country"),
    col("leg.durationInMinutes").alias("duration_in_minutes"),
    col("leg.isSmallestStops").alias("is_smallest_stops"),
    col("leg.timeDeltaInDays").alias("time_delta_days"),
    col("leg.departure").alias("departure_time"),
    col("leg.arrival").alias("arrival_time"),
    col("leg.carriers.operationType").alias("company_operation_type"),
    col("segments.flightNumber").alias("flight_number"),
    col("segments.marketingCarrier.name").alias("marketing_company_name"),
    col("segments.marketingCarrier.alternateId").alias("marketing_company_code"),
    col("segments.operatingCarrier.name").alias("operating_company_name"),
    col("segments.operatingCarrier.alternateId").alias("operating_company_code"),
    col("isSelfTransfer"),
    col("isProtectedSelfTransfer"),
    col("farePolicy.isChangeAllowed"),
    col("farePolicy.isPartiallyChangeable"),
    col("farePolicy.isCancellationAllowed"),
    col("farePolicy.isPartiallyRefundable")
]

# Crear el DataFrame final con los datos limpios y estructurados
final_legs_df = legs_df.select(*selected_columns)

# Guardar en formato parquet en S3 (descomentar si es necesario)
# output_flights_path = "s3://my-aws-project-v1/transformed/skyscanner/flights"
# final_legs_df.write.format("parquet").mode("overwrite").partitionBy("ori", "dest", "year", "month", "week", "day").save(output_flights_path)


In [24]:
final_legs_df.show()

+---+----+-----+----+---+---------+--------------------+-----------+--------------+-----------+--------------+------------------+-------------------+----------------+-------------------+-------------------+-----------------+---------------+-------------------+-------------------+----------------------+-------------+----------------------+----------------------+----------------------+----------------------+--------------+-----------------------+---------------+---------------------+---------------------+---------------------+
|ori|dest|month|week|day|price_raw|              leg_id|origin_name|origin_airport|origin_city|origin_country|  destination_name|destination_airport|destination_city|destination_country|duration_in_minutes|is_smallest_stops|time_delta_days|     departure_time|       arrival_time|company_operation_type|flight_number|marketing_company_name|marketing_company_code|operating_company_name|operating_company_code|isSelfTransfer|isProtectedSelfTransfer|isChangeAllowed|isPar

In [25]:
spark.stop()